# Weighted Average Basic Recommendations

In [1]:
import numpy as np
import pandas as pd

In [2]:
def change_name(x):
    y = x.split(", The")
    if(len(y)==1):
        return x
    else:
        y = "".join([word for word in y])
        y = "The " + y
        return y

In [3]:
ratings = pd.read_csv("../../Datasets/ratings.csv")
movies = pd.read_csv("../../Datasets/movies.csv")
movies['title'] = movies['title'].apply(lambda x: change_name(x))

In [5]:
ratings = ratings.merge(movies)
ratings.dropna(thresh=10, inplace=True, axis=1)
ratings.fillna(0,axis=1)

grouped_ratings = ratings.groupby(by='movieId')['rating'].agg(['mean','count']).rename(columns={'mean':'average_rating','count':'vote_count'})
grouped_ratings = grouped_ratings[grouped_ratings['vote_count']>100]
grouped_ratings['standardized_vote_count'] = (5/grouped_ratings['vote_count'].max())*grouped_ratings['vote_count']
grouped_ratings[['average_rating','standardized_vote_count']].merge(movies,on='movieId').to_csv("../../Datasets/average_ratings.csv")

In [8]:
grouped_ratings['weighted average'] = 0.5*grouped_ratings['average_rating'] + 0.5*grouped_ratings['standardized_vote_count']
grouped_ratings = grouped_ratings['weighted average'].sort_values(ascending=False)
# final_movies = movies.set_index('movieId').loc[grouped_ratings['weighted average'].sort_values(ascending=False).index.values,:].merge(grouped_ratings['weighted average'], on='movieId')
# final_movies.head(20)
movies.set_index('movieId').loc[grouped_ratings.index.values,:].merge(grouped_ratings, on='movieId')

,Unnamed: 0,title,genres,weighted average
movieId,,,,
318,277,The Shawshank Redemption (1994),Crime|Drama,4.623326
356,314,Forrest Gump (1994),Comedy|Drama|Romance|War,4.582067
296,257,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.431361
2571,1939,The Matrix (1999),Action|Sci-Fi|Thriller,4.208685
593,510,The Silence of the Lambs (1991),Crime|Horror|Thriller,4.200706
...,...,...,...,...
39,35,Clueless (1995),Comedy|Romance,2.436908
586,504,Home Alone (1990),Children|Comedy,2.379304
185,156,The Net (1995),Action|Crime|Thriller,2.371153


In [11]:
movies.head()
movies['genres list'] = movies['genres'].str.split(' ')
movies.head()
movies['genres count'] = movies.apply(lambda x: len(set(x['genres list']).intersection('fav_genres')), axis=1)
recommendations = movies[movies['genres count']>0]

ratings['genres list'] = ratings['genres'].str.split(' ')
all_genres_list = []

for i in range(len(movies)):
    all_genres_list.extend(movies.iloc[i]['genres list'])

list(set(all_genres_list))

['Documentary|War',
 'Animation|Drama|War',
 'Action|Animation|Drama',
 'Action|Drama|Horror|Sci-Fi|Thriller',
 'Adventure|Drama|War',
 'Children|Comedy|Crime|Musical',
 'Comedy|Fantasy|Mystery',
 'Animation|Children|Comedy|IMAX',
 'Action|Crime|Thriller|Western',
 'Comedy|Drama|Musical|IMAX',
 'Action|Crime|Romance|Thriller',
 'Romance|War',
 'Action|Animation|Children',
 'Action|Horror|Sci-Fi|Thriller',
 'Action|Adventure|Fantasy|Romance|IMAX',
 'Comedy|Drama|Musical|Romance',
 'Adventure|Crime|Drama|Romance',
 'Action|Comedy|Crime|Drama|Sci-Fi',
 'Drama|Romance|Sci-Fi|Thriller',
 'Action|Animation|Crime|Drama|Film-Noir|Mystery|Sci-Fi|Thriller',
 'Action|Adventure|Fantasy',
 'Fantasy|Horror',
 'Action|Adventure|Children',
 'Animation|Comedy|Drama|Romance|Sci-Fi',
 'Adventure|Animation|Children|Drama|Sci-Fi',
 'Adventure|Documentary|IMAX',
 'Action|Comedy|Horror',
 'Animation|Comedy|Drama|Romance',
 'Drama|Sci-Fi|IMAX',
 'Adventure|Comedy|Crime',
 'Action|Mystery|Sci-Fi|Thriller',
 'C